In [1]:
import os
import sys
import time
from datetime import date
from datetime import datetime
import time
import json
import platform

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)
#//*** tokenize_series(input_series)
#//*** remove_stop_words(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#subreddits = ["wallstreetbets", "stocks", "wallstreetbetsOGs", "spacs", "investing", "pennystocks", "stockmarket", "options", "robinhoodpennystocks", "wallstreetbetsnew", "smallstreetbets"]
filepath = "./data/"
#filename_suffix = "_comments.csv.zip"
#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [2]:
#//*** Path to file aggregate
input_filename = "./data/processed_reddit_v4_1min.csv.zip"

input_filename = "./data/processed_reddit_v4_1min_test.pkl"

raw_df = pd.read_pickle(input_filename)

del raw_df['created_utc']
raw_df

,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
1,No he’s just being sarcastic,2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
2,I was in contact with Schwab and they weren't ...,2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
3,Reelection is never a concern for the communis...,2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
4,"For American brokerages, up to $500,000 are SI...",2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
...,...,...,...,...,...,...,...
23585,"I thought when ETFs deviate from true value, A...",2019-08-27 12:57:00,3.7116,3.7170,3.7116,3.715,9994
23586,"Ditto. I own Hexo - the branding, marketing, d...",2019-08-27 12:58:00,3.7150,3.7150,3.7150,3.715,900
23587,Ugh I know! I’ve been patiently waiting for a ...,2019-08-27 12:59:00,3.7100,3.7168,3.7100,3.715,545
23588,Same here on ACB. Kind of bagholding at this p...,2019-08-27 12:59:00,3.7100,3.7168,3.7100,3.715,545


In [3]:
#//************************************************
#//*** Combine Body Text for each Stock Time
#//************************************************
tdf = pd.DataFrame()

print(len(raw_df['time'].unique()))
print("Starting Body Aggregation....")
start_time = time.time()
for group in raw_df.groupby('time'):
    
    #//*** Build aa temporary Series by grabbing the first line of the group[1] dataframe
    #//*** Only Body will update, the rest are static
    loop_series = group[1].iloc[0].copy()

    
    #//*** Combine all body strings into a single string, add to loop_series['body']
    loop_series['body'] = " ".join(list(group[1]['body']))
    #//*** Build a dataframe to hold the Series.
    #//*** Add Dataframe to tdf
    #print(loop_series)
    tdf = pd.concat ([tdf,pd.DataFrame(data=[loop_series], columns = list(group[1].columns))])

print(f"Built: {round(time.time()-start_time,2)}")
tdf

5010
Starting Body Aggregation....
Built: 10.18


,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.0900,4.0900,4.0900,4.090,406
11,It’s a combination of options but I made big o...,2019-07-24 09:35:00,4.1400,4.1500,4.1300,4.150,10116
12,Idk about you but I know several people with H...,2019-07-24 09:36:00,4.1400,4.1700,4.1385,4.170,16449
13,Thanks.. Will check them out,2019-07-24 09:37:00,4.1700,4.1800,4.1540,4.175,15605
14,I'm waiting until Monday to sell a covered cal...,2019-07-24 09:40:00,4.1900,4.1950,4.1700,4.180,42314
...,...,...,...,...,...,...,...
23584,Which stock are you referencing?,2019-08-27 12:56:00,3.7150,3.7150,3.7100,3.710,416
23585,"I thought when ETFs deviate from true value, A...",2019-08-27 12:57:00,3.7116,3.7170,3.7116,3.715,9994
23586,"Ditto. I own Hexo - the branding, marketing, d...",2019-08-27 12:58:00,3.7150,3.7150,3.7150,3.715,900
23587,Ugh I know! I’ve been patiently waiting for a ...,2019-08-27 12:59:00,3.7100,3.7168,3.7100,3.715,545


In [4]:
#//************************************************
#//*** Clean text body
#//************************************************

tdf['token'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(tdf['body'],{"remove_empty":False})))
print("Done")

#//*** Detokenize the clean column as tfidf
tdf['tfidf'] = tdf['token'].apply(lambda x: ' '.join(x))


remove_empty False
Text Cleaning Time: 0.6084105968475342
Tokenize Time: 1.5309710502624512
Stop Words Time: 0.6312947273254395
Done


In [6]:
#//*** Build tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#loop_list.append(tfidf.fit_transform(input_df['tfidf']))
tfidf_matrix = []
tfidf_list = []
tfidf = TfidfVectorizer()

print("Starting tfidf....")
start_time = time.time()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(tdf['tfidf'])


print(f"Built: {round(time.time()-start_time,2)}")

print(tfidf_matrix)

Starting tfidf....
Built: 0.33
  (0, 8133)	0.08745808745100456
  (0, 11133)	0.07327893552266958
  (0, 2875)	0.05745903186031536
  (0, 12858)	0.05009304185394831
  (0, 1271)	0.059828786385647564
  (0, 15883)	0.06475547596357362
  (0, 14146)	0.04850398207814858
  (0, 15909)	0.0967335584413286
  (0, 12160)	0.055511687856221854
  (0, 10623)	0.037181240474684976
  (0, 14506)	0.04293238130669861
  (0, 10000)	0.051679345634732955
  (0, 11281)	0.05792547524640468
  (0, 17464)	0.03648588477406036
  (0, 9638)	0.03109594680876921
  (0, 6328)	0.05332397563086158
  (0, 5387)	0.08913682801099229
  (0, 14747)	0.05538498395581354
  (0, 5053)	0.06446319823408891
  (0, 3136)	0.08154009758065602
  (0, 3067)	0.03470696153058779
  (0, 5639)	0.036828048837588646
  (0, 15687)	0.03635195522166141
  (0, 9140)	0.0552597281316865
  (0, 9863)	0.04670937342947679
  :	:
  (5008, 8008)	0.21328624075614527
  (5008, 12039)	0.17110212787774884
  (5008, 6375)	0.14488634710955123
  (5008, 15717)	0.12363931585752246
  (50

In [8]:
import pickle
output_filename = "./data/processed_reddit_v4_1min_testdata_tfidf.pkl"

with open(output_filename, 'wb') as f:
    pickle.dump(tfidf, f)

output_filename = "./data/processed_reddit_v4_1min_testdata_tfidf_matrix.pkl"
with open(output_filename, 'wb') as f:
    pickle.dump(tfidf_matrix, f)


In [18]:
output_filename = "./data/processed_reddit_v4_1min_amc_stock_comments_test.pkl"

#//*** Cleanup columns for storage
if 'body' in tdf.columns:
    del tdf['body']
    del tdf['tfidf']

tdf.to_pickle(output_filename)

In [17]:
output_filename = "./data/processed_reddit_v4_1min_testdata_tfidf.pkl"
infile = open(output_filename,'rb')
t_tdif = pickle.load(infile)
infile.close()
print(t_tdif.get_params())
print(t_tdif.get_feature_names())


{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [10]:
infile = open(output_filename,'rb')
t_tdif_matrix = pickle.load(infile)
infile.close()
print(t_tdif_matrix)

  (0, 8133)	0.08745808745100456
  (0, 11133)	0.07327893552266958
  (0, 2875)	0.05745903186031536
  (0, 12858)	0.05009304185394831
  (0, 1271)	0.059828786385647564
  (0, 15883)	0.06475547596357362
  (0, 14146)	0.04850398207814858
  (0, 15909)	0.0967335584413286
  (0, 12160)	0.055511687856221854
  (0, 10623)	0.037181240474684976
  (0, 14506)	0.04293238130669861
  (0, 10000)	0.051679345634732955
  (0, 11281)	0.05792547524640468
  (0, 17464)	0.03648588477406036
  (0, 9638)	0.03109594680876921
  (0, 6328)	0.05332397563086158
  (0, 5387)	0.08913682801099229
  (0, 14747)	0.05538498395581354
  (0, 5053)	0.06446319823408891
  (0, 3136)	0.08154009758065602
  (0, 3067)	0.03470696153058779
  (0, 5639)	0.036828048837588646
  (0, 15687)	0.03635195522166141
  (0, 9140)	0.0552597281316865
  (0, 9863)	0.04670937342947679
  :	:
  (5008, 8008)	0.21328624075614527
  (5008, 12039)	0.17110212787774884
  (5008, 6375)	0.14488634710955123
  (5008, 15717)	0.12363931585752246
  (5008, 7543)	0.18153131157240643
 